In [2]:
import pandas as pd

# Nettoyage des données

In [8]:
df_train = pd.read_csv("C:\\Users\\user\\Documents\\GitHub\\immo_TD\\Data\\data_train.csv", sep=",")

In [9]:
#On enleve la colonne Normalized_Mutations
df_train = df_train.drop(columns=['Normalized_Mutations'])

In [10]:
df_train.head(2)

,Valeur fonciere,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain,exterieur,year,month,nom_region,pop_active,salaire_moyen,nb_etab_elem,mean_prixm2,q1_prixm2,q3_prixm2,Total_Mutations
0,90000.0,Maison,150.0,3,347.0,1.0,2018,1,Auvergne-Rhône-Alpes,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389
1,154200.0,Maison,180.0,6,680.0,1.0,2018,6,Auvergne-Rhône-Alpes,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389


In [11]:
# Dummies Variable REGION
dummies = pd.get_dummies(df_train['nom_region'], prefix='Region')
# Concaténez les variables indicatrices avec le DataFrame d'origine
df_train = pd.concat([df_train, dummies], axis=1)
#Drop colonne Nom_Region
df_train = df_train.drop(columns=['nom_region'])

In [12]:
#On split le df en 4df suivant le Type Local
df_local=df_train.loc[df_train['Type local']=='Local']
df_dependance=df_train.loc[df_train['Type local']=='Dépendance']
df_maison=df_train.loc[df_train['Type local']=='Maison']
df_appartement=df_train.loc[df_train['Type local']=='Appartement']

In [13]:
print("taille local",df_local.shape)
print("taille dependance",df_dependance.shape)
print("taille maison",df_maison.shape)
print("taille appartement",df_appartement.shape)

taille local (54448, 28)
taille dependance (164546, 28)
taille maison (931324, 28)
taille appartement (469785, 28)


In [14]:
df_local.head()

,Valeur fonciere,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain,exterieur,year,month,pop_active,salaire_moyen,...,Region_Centre-Val de Loire,Region_Corse,Region_Grand Est,Region_Hauts-de-France,Region_Normandie,Region_Nouvelle-Aquitaine,Region_Occitanie,Region_Pays de la Loire,Region_Provence-Alpes-Côte d'Azur,Region_Île-de-France
29,78300.0,Local,0.0,0,783.0,1.0,2018,1,320682.0,15.336249,...,0,0,0,0,0,0,0,0,0,0
74,55000.0,Local,75.0,0,0.0,0.0,2018,1,320682.0,15.336249,...,0,0,0,0,0,0,0,0,0,0
75,55000.0,Local,75.0,0,0.0,0.0,2018,1,320682.0,15.336249,...,0,0,0,0,0,0,0,0,0,0
86,110000.0,Local,98.0,0,0.0,0.0,2018,1,320682.0,15.336249,...,0,0,0,0,0,0,0,0,0,0
87,110000.0,Local,98.0,0,0.0,0.0,2018,1,320682.0,15.336249,...,0,0,0,0,0,0,0,0,0,0


## Mise en modele

### Arbre de regression LOCAL

In [23]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

In [117]:
y=df_local["Valeur fonciere"]
X = df_local.drop(columns=['Valeur fonciere',"Type local"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [118]:
# Créez un modèle d'arbre de régression
regression_tree = DecisionTreeRegressor()

# Entraînez le modèle sur vos données d'entraînement
regression_tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [119]:
# Faites des prédictions sur de nouvelles données  # Remplacez par vos caractéristiques
y_pred= regression_tree.predict(X_test)


In [121]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
# Calcul de l'erreur quadratique moyenne (RMSE)
rmse = mean_squared_error(y_test, y_pred)

# Calcul de l'erreur absolue moyenne (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calcul du coefficient de détermination (R2)
r2 = r2_score(y_test, y_pred)

# Affichage des résultats
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R-squared (R2): {r2}')


MSE: 6343326446.937833
MAE: 58807.42832908662
R-squared (R2): -0.10861796149828629


In [83]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [122]:
param_grid = {
    'criterion': ['mse'],  # Critère de fractionnement
    'max_depth': [None, 10, 20],  # Profondeur maximale de l'arbre
    'min_samples_split': [50, 100, 200],  # Nombre minimum d'échantillons pour scinder un nœud
    'min_samples_leaf': [25, 50, 100]  # Nombre minimum d'échantillons par feuille
}

In [123]:
regression_tree = DecisionTreeRegressor()
grid_search = GridSearchCV(regression_tree, param_grid, cv=None, scoring='neg_mean_squared_error')

In [124]:
grid_search.fit(X_train, y_train)


c:\Users\user\anaconda3\lib\site-packages\sklearn\tree\_classes.py:359: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\tree\_classes.py:359: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\tree\_classes.py:359: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\tree\_classes.py:359: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\tree\_classes.py:359: 

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['mse'], 'max_depth': [None, 10, 20],
                         'min_samples_leaf': [25, 50, 100],
                         'min_samples_split': [50, 100, 200]},
             scoring='neg_mean_squared_error')

In [126]:
best_params = grid_search.best_params_
print(best_params)
best_regressor = grid_search.best_estimator_
print(best_regressor)

{'criterion': 'mse', 'max_depth': 20, 'min_samples_leaf': 50, 'min_samples_split': 200}
DecisionTreeRegressor(criterion='mse', max_depth=20, min_samples_leaf=50,
                      min_samples_split=200)


In [129]:
from sklearn.tree import DecisionTreeRegressor

# Créez un nouvel arbre de régression avec les paramètres modifiés
new_regressor = DecisionTreeRegressor(criterion='mse', max_depth=20, min_samples_leaf=50, min_samples_split=200)

# Entraînez le modèle avec de nouvelles données (X_train, y_train)
new_regressor.fit(X_train, y_train)

# Faites des prédictions avec le nouveau modèle
y_pred = new_regressor.predict(X_test)


c:\Users\user\anaconda3\lib\site-packages\sklearn\tree\_classes.py:359: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(


In [130]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
# Calcul de l'erreur quadratique moyenne (RMSE)
rmse = mean_squared_error(y_test, y_pred)

# Calcul de l'erreur absolue moyenne (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calcul du coefficient de détermination (R2)
r2 = r2_score(y_test, y_pred)

# Affichage des résultats
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R-squared (R2): {r2}')


MSE: 6343326446.937833
MAE: 58873.97019595509
R-squared (R2): 0.22820777065368925


## APPARTEMENT

In [97]:
y=df_appartement["Valeur fonciere"]
X = df_appartement.drop(columns=['Valeur fonciere',"Type local"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [98]:
# Créez un modèle d'arbre de régression
regression_tree = DecisionTreeRegressor()

# Entraînez le modèle sur vos données d'entraînement
regression_tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [99]:
# Faites des prédictions sur de nouvelles données  # Remplacez par vos caractéristiques
y_pred= regression_tree.predict(X_test)

In [106]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
# Calcul de l'erreur quadratique moyenne (RMSE)
mse = mean_squared_error(y_test, y_pred)

# Calcul de l'erreur absolue moyenne (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calcul du coefficient de détermination (R2)
r2 = r2_score(y_test, y_pred)

# Affichage des résultats
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R-squared (R2): {r2}')


MSE: 4706022676.35541
MAE: 45537.955845521195
R-squared (R2): 0.33392387928594724


## MAISON

In [108]:
y=df_maison["Valeur fonciere"]
X = df_maison.drop(columns=['Valeur fonciere',"Type local"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [109]:
# Créez un modèle d'arbre de régression
regression_tree = DecisionTreeRegressor()

# Entraînez le modèle sur vos données d'entraînement
regression_tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [110]:
# Faites des prédictions sur de nouvelles données  # Remplacez par vos caractéristiques
y_pred= regression_tree.predict(X_test)

In [111]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
# Calcul de l'erreur quadratique moyenne (RMSE)
mse = mean_squared_error(y_test, y_pred)

# Calcul de l'erreur absolue moyenne (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calcul du coefficient de détermination (R2)
r2 = r2_score(y_test, y_pred)

# Affichage des résultats
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R-squared (R2): {r2}')


MSE: 7668223929.968928
MAE: 56849.74816908336
R-squared (R2): 0.17173839265601643


##  DEPENDANCE

In [112]:
y=df_dependance["Valeur fonciere"]
X = df_dependance.drop(columns=['Valeur fonciere',"Type local"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [113]:
# Créez un modèle d'arbre de régression
regression_tree = DecisionTreeRegressor()

# Entraînez le modèle sur vos données d'entraînement
regression_tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [114]:
# Faites des prédictions sur de nouvelles données  # Remplacez par vos caractéristiques
y_pred= regression_tree.predict(X_test)

In [115]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
# Calcul de l'erreur quadratique moyenne (RMSE)
mse = mean_squared_error(y_test, y_pred)

# Calcul de l'erreur absolue moyenne (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calcul du coefficient de détermination (R2)
r2 = r2_score(y_test, y_pred)

# Affichage des résultats
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R-squared (R2): {r2}')


MSE: 6343326446.937833
MAE: 58004.89804530851
R-squared (R2): 0.19450053970924341
